In [1]:
# https://github.com/DataTalksClub/llm-zoomcamp/blob/main/03-vector-search/eval/evaluation-metrics.md

In [2]:
2+2

4

In [3]:
import pandas as pd
from tqdm.auto import tqdm
import json
import sys
import os
from dotenv import load_dotenv
# Load environment variables from .env
load_dotenv("/home/jovyan/.envrc")

True

In [4]:
from elasticsearch import Elasticsearch

# Create an instance of the Elasticsearch client
es = Elasticsearch(['http://elasticsearch:9200'])

# Check if the connection is established
if es.ping():
    print("Elasticsearch is connected!")
else:
    print("Elasticsearch is not reachable.")

Elasticsearch is connected!


In [5]:
from sentence_transformers import SentenceTransformer

In [6]:
from elasticsearch import Elasticsearch

In [7]:
# # Directory containing the data files
# data_dir = os.path.abspath('../reviews-assistant/data/reviews')

# # Initialize an empty list to hold all reviews
# reviews = []

# # List objects in the directory
# objects_in_directory = os.listdir(data_dir)

# # Iterate over the files in the directory
# for obj in objects_in_directory:
#     if obj.endswith('.json'):  # Check if the file is a JSON file
#         file_path = os.path.join(data_dir, obj)
#         with open(file_path, 'r', encoding='utf-8') as jsonfile:
#             # Load the reviews from the JSON file
#             file_reviews = json.load(jsonfile)
#             reviews.extend(file_reviews)  # Append reviews to the main list
# # Print the first i reviews
# i = 2  # Change this to print more reviews if needed
# for review in reviews[:i]:
#     print(f"Author ID: {review['author.steamid']}")
#     print(f"Review: {review.get('review', 'No text')}")
#     print(f"Timestamp Created: {review['timestamp_created']}")
#     print("-" * 79)

In [8]:
# len(reviews)

In [9]:
# reviews[1:2]

In [10]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

/opt/conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
# for doc in tqdm(reviews):
#     title = doc["title"]
#     review = doc['review']
#     tr = title + ' ' + review

#     doc['title_vector'] = model.encode(title)
#     doc['review_vector'] = model.encode(review)
#     doc['title_review_vector'] = model.encode(tr)

In [12]:
# doc.keys()

In [13]:
from elasticsearch import Elasticsearch, ConnectionError

try:
    es = Elasticsearch(['http://elasticsearch:9200'])  # Use the service name
    if es.ping():
        print("Connected to Elasticsearch")
    else:
        print("Could not connect to Elasticsearch")
except ConnectionError as e:
    print(f"Connection error: {e}")


Connected to Elasticsearch


In [14]:
from elasticsearch import Elasticsearch, NotFoundError, ConnectionError
from tqdm import tqdm

class ReviewIndexer:
    def __init__(self, es_host='http://elasticsearch:9200', index_name='steam-reviews', model=None):
        self.es = Elasticsearch([es_host])
        self.index_name = index_name
        self.model = model  # Expecting a model to encode text
        
        # Check the connection upon initialization
        self.check_connection()
        
        self.index_settings = {
            "settings": {
                "number_of_shards": 1,
                "number_of_replicas": 0
            },
            "mappings": {
                "properties": {
                    "appid": {"type": "keyword"},
                    "timestamp_query": {"type": "integer"},
                    "title": {"type": "keyword"},
                    "recommendationid": {"type": "keyword"},  # Changed to keyword for consistency
                    "author.steamid": {"type": "keyword"},
                    "author.playtimeforever": {"type": "integer"},
                    "author.playtime_last_two_weeks": {"type": "integer"},
                    "author.playtime_at_review": {"type": "integer"},
                    "author.last_played": {"type": "integer"},
                    "language": {"type": "keyword"},
                    "review": {"type": "text"},
                    "voted_up": {"type": "boolean"},  # Changed to boolean for consistency
                    "votes_up": {"type": "integer"},
                    "timestamp_created": {"type": "integer"},
                    "timestamp_updated": {"type": "integer"},
                    "title_vector": {
                        "type": "dense_vector",
                        "dims": 384,
                        "index": True,
                        "similarity": "cosine"
                    },
                    "review_vector": {
                        "type": "dense_vector",
                        "dims": 384,
                        "index": True,
                        "similarity": "cosine"
                    },
                    "title_review_vector": {
                        "type": "dense_vector",
                        "dims": 384,
                        "index": True,
                        "similarity": "cosine"
                    },
                }
            }
        }

    def check_connection(self):
        try:
            if not self.es.ping():
                raise ConnectionError("Elasticsearch cluster is down!")
            print("Successfully connected to Elasticsearch.")
        except ConnectionError as e:
            print(f"Connection error: {e}")
            raise

    def create_index(self):
        try:
            self.es.indices.delete(index=self.index_name, ignore_unavailable=True)
            print(f"Deleted index: {self.index_name} (if it existed)")
        except NotFoundError:
            print(f"Index {self.index_name} does not exist, nothing to delete.")
        except ConnectionError as e:
            print(f"Error deleting index: {e}")

        try:
            self.es.indices.create(index=self.index_name, body=self.index_settings)
            print(f"Created index: {self.index_name}")
        except ConnectionError as e:
            print(f"Error creating index: {e}")

    def prepare_and_index_documents(self, reviews):
        """
        Prepare documents by encoding and indexing them into Elasticsearch.
        
        :param reviews: List of review documents to be indexed.
        """
        for doc in tqdm(reviews):
            # Handle None values for numerical fields
            doc['author.playtimeforever'] = doc.get('author.playtimeforever', 0) or 0
            doc['author.playtime_last_two_weeks'] = doc.get('author.playtime_last_two_weeks', 0) or 0
            doc['author.playtime_at_review'] = doc.get('author.playtime_at_review', 0) or 0
            doc['author.last_played'] = doc.get('author.last_played', 0) or 0
            doc['votes_up'] = doc.get('votes_up', 0) or 0
            doc['timestamp_created'] = doc.get('timestamp_created', 0) or 0
            doc['timestamp_updated'] = doc.get('timestamp_updated', 0) or 0

            # Encode title and review if a model is provided
            if self.model:
                title = doc["title"]
                review = doc['review']
                tr = title + ' ' + review

                # Encode the text and convert to lists
                doc['title_vector'] = self.model.encode(title).tolist()
                doc['review_vector'] = self.model.encode(review).tolist()
                doc['title_review_vector'] = self.model.encode(tr).tolist()
            else:
                # Placeholder if no model is provided
                doc['title_vector'] = []
                doc['review_vector'] = []
                doc['title_review_vector'] = []

            # Index the document
            try:
                self.es.index(index=self.index_name, document=doc)
            except Exception as e:
                print(f"Failed to index document {doc}: {e}")

    def fetch_first_documents(self, size=10):
        body = {
            "query": {
                "match_all": {}
            },
            "size": size
        }

        try:
            response = self.es.search(index=self.index_name, body=body)
            return response['hits']['hits']
        except NotFoundError:
            print(f"Index {self.index_name} not found.")
        except ConnectionError as e:
            print(f"Error searching index: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")
        return None

    
    def query_reviews(self, title, query, k=5, num_candidates=10000):
        """
        Query reviews using both KNN and keyword search in a single query, 
        where both conditions are part of the search request.
        
        :param title: The title of the game to filter results.
        :param query: The natural language query for searching.
        :param k: The number of nearest neighbors to retrieve.
        :param num_candidates: Number of candidates for KNN search.
        :return: The combined search results from KNN and keyword queries.
        """
        # Encode the query into a vector
        v_q = self.model.encode(query).tolist()  # Ensure the query is encoded into a list
        
        # KNN query
        knn_query = {
            "field": "review_vector",  # Field for KNN vector search
            "query_vector": v_q,
            "k": k,
            "num_candidates": num_candidates,
            "boost": 0.5,
            "filter": {
                "term": {
                    "title": title  # Filter by title field
                }
            }
        }
    
        # Keyword search query using bool/must
        keyword_query = {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["review^3", "title"],  # Relevant fields for keyword search
                        "type": "best_fields",
                        "boost": 0.5
                    }
                },
                "filter": {
                    "term": {
                        "title": title  # Filter by title field
                    }
                }
            }
        }
    
        # Combining both knn and keyword queries into one search query
        search_query = {
            "knn": knn_query,
            "query": keyword_query,
            "size": k,
            "_source": ["review", "title", "author.steamid", "votes_up", "timestamp_created"]
        }
    
        # Execute the search query
        try:
            es_results = self.es.search(index=self.index_name, body=search_query)
            result_docs = [hit['_source'] for hit in es_results['hits']['hits']]
            return result_docs
        except Exception as e:
            print(f"An error occurred during the search: {e}")
            return []
    
    
    def query_reviews_hybrid(self, appid, query, v_q, k=2, num_candidates=10000, vector_field="review_vector"):
        """
        Query reviews using both KNN and keyword search in a single query with hybrid search, 
        where both conditions are part of the search request.
        
        :param title: The title of the game to filter results.
        :param query: The natural language query for searching.
        :param k: The number of nearest neighbors to retrieve.
        :param num_candidates: Number of candidates for KNN search.
        :param vector_field: The field to use for KNN vector search. Can be 'review_vector', 'title_vector', or 'title_review_vector'.
        :return: The combined search results from KNN and keyword queries.
        """
        
        # Validate the vector field
        if vector_field not in ["review_vector", "title_vector", "title_review_vector"]:
            raise ValueError(f"Invalid vector field: {vector_field}. Must be 'review_vector', 'title_vector', or 'title_review_vector'.")
        
        # Encode the query into a vector
        # v_q = self.model.encode(query).tolist()  # Ensure the query is encoded into a list
        
        # KNN query
        knn_query = {
            "field": vector_field,  # Use the selected vector field
            "query_vector": v_q,
            "k": k,
            "num_candidates": num_candidates,
            "boost": 0.5,
            "filter": {
                "term": {
                    "appid": appid  # Filter by title field
                }
            }
        }
    
        # Keyword search query using bool/must
        keyword_query = {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["review^3", "appid"],  # Relevant fields for keyword search
                        "type": "best_fields",
                        "boost": 0.5
                    }
                },
                "filter": {
                    "term": {
                        "appid": appid  # Filter by title field
                    }
                }
            }
        }
    
        # Combining both knn and keyword queries into one search query
        search_query = {
            "knn": knn_query,
            "query": keyword_query,
            "size": k,
            "_source": ["review", "appid"]
        }
    
        # Execute the search query
        try:
            es_results = self.es.search(index=self.index_name, body=search_query)
            result_docs = [hit['_source'] for hit in es_results['hits']['hits']]
            return result_docs
        except Exception as e:
            print(f"An error occurred during the search: {e}")
            return []




    def get_distinct_titles(self):
        """
        Get distinct titles from the Elasticsearch index.
    
        :return: A list of distinct titles.
        """
        try:
            # Define the aggregation query
            aggs = {
                "distinct_titles": {
                    "terms": {
                        "field": "title.keyword",  # Use ".keyword" for exact matches on keyword fields
                        "size": 1000  # Adjust this size as needed
                    }
                }
            }
    
            # Execute the aggregation query
            response = self.es.search(index=self.index_name, body={"size": 0, "aggs": aggs})
            distinct_titles = response['aggregations']['distinct_titles']['buckets']
    
            # Extract titles from the buckets
            titles_list = [bucket['key'] for bucket in distinct_titles]
            return titles_list
    
        except Exception as e:
            print(f"An error occurred while fetching distinct titles: {e}")
            return []


# Usage example
# Assuming you have a list of reviews and a model for encoding
indexer = ReviewIndexer(model=model)  # Pass your model here
indexer.create_index()  # Ensure the index is created first

# Sample input for indexing
# reviews = [
#     {
#         'appid': '721180',
#         'timestamp_query': 1727608376,
#         'title': 'Dustborn',
#         'recommendationid': '173934401',
#         'author.steamid': '76561198028269344',
#         'author.playtimeforever': None,
#         'author.playtime_last_two_weeks': 0,
#         'author.playtime_at_review': 90,
#         'author.last_played': 1725140143,
#         'language': 'english',
#         'review': 'I went into this with an open mind...',
#         'voted_up': False,
#         'votes_up': 401,
#         'timestamp_created': 1725142580,
#         'timestamp_updated': 1725142580,
#         'title_vector': [],  # Initial placeholders
#         'review_vector': [],
#         'title_review_vector': []
#     }
# ]

# # Prepare and index documents
# indexer.prepare_and_index_documents(reviews)

# # Fetch and print the first documents after indexing
# first_documents = indexer.fetch_first_documents(size=10)
# if first_documents:
#     for doc in first_documents:
#         print(doc['_source'])  # Print the document details


Successfully connected to Elasticsearch.
Deleted index: steam-reviews (if it existed)
Created index: steam-reviews


In [15]:
# Directory containing the data files
data_dir = os.path.abspath('../reviews-assistant/data/reviews')

# Initialize an empty list to hold all reviews
reviews = []

# List objects in the directory
objects_in_directory = os.listdir(data_dir)

# Iterate over the files in the directory
for obj in objects_in_directory:
    if obj.endswith('.json'):  # Check if the file is a JSON file
        file_path = os.path.join(data_dir, obj)
        with open(file_path, 'r', encoding='utf-8') as jsonfile:
            # Load the reviews from the JSON file
            file_reviews = json.load(jsonfile)
            reviews.extend(file_reviews)  # Append reviews to the main list
# Print the first i reviews
i = 2  # Change this to print more reviews if needed
for review in reviews[:i]:
    print(f"Author ID: {review['author.steamid']}")
    print(f"Review: {review.get('review', 'No text')}")
    print(f"Timestamp Created: {review['timestamp_created']}")
    print("-" * 79)

Author ID: 76561198420943538
Review: ---{ Graphics }---
✅ You forget what reality is
☐ Beautiful
☐ Good
☐ Decent
☐ Bad
☐ You will get eye cancer
☐ Get a pepper spray for your eye instead

---{ Gameplay }---
☐ Won’t ever touch any other game anymore
✅ Very good
☐ Good
☐ It's just gameplay
☐ Mehh
☐ Watch paint dry instead
☐ Tic Tac toe is better

---{ Audio }---
☐ Eargasm
✅ Very good
☐ Good
☐ Not too bad
☐ Bad
☐ I'm now deaf

---{ Audience }---
☐ Kids
✅Teens
✅ Adults
☐ Grandma

---{ PC Requirements }---
☐ Check if you can run paint
☐ Potato
☐ Decent
✅ Fast
☐ Rich boi
☐ Ask NASA if they have a spare computer
☐ Search the galaxy for dark matter fuel to run

---{ Difficulty }---
☐ Just press 'W'
☐ Easy
✅ Easy to learn / Hard to master
☐ Significant brain usage
☐ Difficult
☐ Dark Souls

---{ Grind }---
☐ Nothing to grind
☐ Only if u care about leaderboards/ranks
✅ Isn't necessary to progress
☐ Average grind level
☐ Too much grind
☐ You'll need a second life for grinding

---{ Story }---
☐ No

In [16]:
len(reviews)

1782

In [17]:
reviews[0]

{'appid': '812140',
 'timestamp_query': 1727643597,
 'title': "Assassin's Creed Odyssey",
 'recommendationid': '174160338',
 'author.steamid': '76561198420943538',
 'author.playtimeforever': None,
 'author.playtime_last_two_weeks': 0,
 'author.playtime_at_review': 5599,
 'author.last_played': 1696845057,
 'language': 'english',
 'review': "---{ Graphics }---\n✅ You forget what reality is\n☐ Beautiful\n☐ Good\n☐ Decent\n☐ Bad\n☐ You will get eye cancer\n☐ Get a pepper spray for your eye instead\n\n---{ Gameplay }---\n☐ Won’t ever touch any other game anymore\n✅ Very good\n☐ Good\n☐ It's just gameplay\n☐ Mehh\n☐ Watch paint dry instead\n☐ Tic Tac toe is better\n\n---{ Audio }---\n☐ Eargasm\n✅ Very good\n☐ Good\n☐ Not too bad\n☐ Bad\n☐ I'm now deaf\n\n---{ Audience }---\n☐ Kids\n✅Teens\n✅ Adults\n☐ Grandma\n\n---{ PC Requirements }---\n☐ Check if you can run paint\n☐ Potato\n☐ Decent\n✅ Fast\n☐ Rich boi\n☐ Ask NASA if they have a spare computer\n☐ Search the galaxy for dark matter fuel to

In [18]:
reviews[-1]

{'appid': '2239550',
 'timestamp_query': 1727643597,
 'title': 'Watch Dogs: Legion',
 'recommendationid': '174862159',
 'author.steamid': '76561199371555316',
 'author.playtimeforever': None,
 'author.playtime_last_two_weeks': 1,
 'author.playtime_at_review': 109,
 'author.last_played': 1726993288,
 'language': 'english',
 'review': 'meh',
 'voted_up': True,
 'votes_up': 0,
 'timestamp_created': 1726233314,
 'timestamp_updated': 1726233314}

In [19]:
# Prepare and index documents
indexer.prepare_and_index_documents(reviews)

100%|██████████| 1782/1782 [02:41<00:00, 11.07it/s]


In [20]:
# print(doc['_source'].keys())

In [21]:
# Fetch and print the first documents after indexing
# first_documents = indexer.fetch_first_documents(size=10)
# if first_documents:
#     for doc in first_documents:
#         print(doc['_source'])  # Print the document details

In [22]:
# # Create an instance of ReviewIndexer
# indexer = ReviewIndexer()

# # Get distinct titles
# distinct_titles = indexer.get_distinct_titles()

# # Print distinct titles
# for title in distinct_titles:
#     print(title)

In [23]:
# results

In [24]:
# # Usage example
# indexer = ReviewIndexer(model=model)  # Ensure your model is passed here
# # indexer.create_index()  # Ensure the index is created first

# # Sample input for querying
# title = "God of War: Ragnarok"  # Example game title
# query = 'Is God of War: Ragnarok a game for kids?'

# # Perform the query
# results = indexer.query_reviews(title, query)

# # Convert the results to a pandas DataFrame
# if results:  # Check if results are not empty
#     # Extract the source data from the Elasticsearch response
#     data = [doc['_source'] for doc in results]
    
#     # Create a DataFrame
#     df_results = pd.DataFrame(data)
    
#     # Set pandas options to display full content of the 'review' column
#     # pd.set_option('display.max_colwidth', None)  # or use a larger number to set a specific width
#     # pd.set_option('display.max_rows', None)  # Show all rows if needed
#     pd.reset_option('all')

#     # Display the DataFrame
#     # print(df_results.head())  # or use df_results to view the entire DataFrame
# else:
#     print("No results found.")

In [25]:
title = "God of War: Ragnarok"  # Example game title
query = 'Is God of War: Ragnarok a game for kids?'

# Perform the query
results = indexer.query_reviews(title, query)

In [26]:
results

[{'title': 'God of War: Ragnarok',
  'author.steamid': '76561198354466742',
  'review': "to God of War: Ragnarok Dev. Team,\n\nI’m writing to bring to your attention an issue regarding the current PC port of God of War: Ragnarok that affects users with 4GB VRAM graphics cards. When attempting to launch the game on such systems, an error message is displayed, indicating that there is not enough VRAM, and the game closes without allowing users to proceed. It is my humble request to kindly reconsider or remove this VRAM limitation. From personal experience and community feedback, I can confirm that the game runs well on certain integrated graphics processors (such as the Ryzen 3 3200G, Ryzen 5 3400G, and Ryzen 7 5700G), which are generally weaker than many 4GB VRAM GPUs available today. God of War: Ragnarok performs on the base PlayStation 4, which further suggests that the game should be capable of running on 4GB VRAM graphics cards, and the latest Steam Hardware Survey shows that one of

In [27]:
# Usage example
# indexer = ReviewIndexer(model=model)  # Ensure your model is passed here

# Sample input for querying
title = "God of War: Ragnarok"  # Example game title
appid = "2322010"
query = 'Is God of War: Ragnarok a game for kids?'

# Perform the query
results = indexer.query_reviews(appid, query)

# Convert the results to a pandas DataFrame
if results:  # Check if results are not empty
    # Since `results` is already a list of dictionaries (docs), no need to extract '_source'
    df_results = pd.DataFrame(results)
    
    # Reset pandas options to default (if necessary)
    pd.reset_option('all')
    
    # Display the DataFrame
    # You can print the DataFrame or use df_results.head() to display the top rows
    print(df_results.head())
else:
    print("No results found.")

No results found.


In [28]:
import pandas as pd

# Initialize ReviewIndexer with the model
# indexer = ReviewIndexer(model=model)  # Ensure your model is passed here
# indexer.create_index()  # Ensure the index is created first

# Sample input for querying
title = "God of War: Ragnarok"  # Example game title
query = 'Is God of War: Ragnarok a game for kids?'
v_q = model.encode(query)

# Perform the query using different vector fields
for vector_field in ["review_vector", "title_vector", "title_review_vector"]:
    print(f"\nQuerying with vector field: {vector_field}")
    results = indexer.query_reviews_hybrid(appid, query, v_q, vector_field=vector_field)

    # Convert the results to a pandas DataFrame
    if results:  # Check if results are not empty
        # Since `results` is already a list of dictionaries (docs), no need to extract '_source'
        df_results = pd.DataFrame(results)

        # Reset pandas options to default (if necessary)
        pd.reset_option('all')

        # Display the DataFrame
        print(df_results.head())  # Display the top rows of the DataFrame
    else:
        print("No results found.")



Querying with vector field: review_vector
     appid                                             review
0  2322010  to God of War: Ragnarok Dev. Team,\n\nI’m writ...
1  2322010  God of War Ragnarok could have been an unforge...

Querying with vector field: title_vector


/tmp/ipykernel_58455/3957917344.py:23: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')
/tmp/ipykernel_58455/3957917344.py:23: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')


     appid                                             review
0  2322010  to God of War: Ragnarok Dev. Team,\n\nI’m writ...
1  2322010  God of War Ragnarok could have been an unforge...

Querying with vector field: title_review_vector
     appid                                             review
0  2322010  to God of War: Ragnarok Dev. Team,\n\nI’m writ...
1  2322010  God of War Ragnarok could have been an unforge...


/tmp/ipykernel_58455/3957917344.py:23: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')


In [29]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [30]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score += 1 / (rank + 1)
                break  # Stop after finding the first relevant document

    return total_score / len(relevance_total)


In [31]:
# def question_hybrid(q):
#     question = q['question']
#     course = q['course']

#     v_q = model.encode(question)

#     return elastic_search_hybrid('question_vector', question, v_q, course)

In [32]:
def question_hybrid(q):
    # title = "God of War: Ragnarok"  # Example game title
    # query = 'Is God of War: Ragnarok a game for kids?'
    query=q['question']
    appid=q['appid']
    v_q = model.encode(query)

    return indexer.query_reviews_hybrid(appid, query, v_q, vector_field="review_vector")

In [33]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        print(q)
        
        # doc_id = q['question']
        doc_id = q['appid']
        
        results = search_function(q)
        print(results[-1])
        
        relevance = [d['appid'] == doc_id for d in results]
        print(relevance[-1])
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [34]:
import json
import os

# Define the data directory path
data_dir = os.path.abspath('../reviews-assistant/data/ground_truth')

# Define the input file path
input_file = os.path.join(data_dir, "ground_truth_retrieval.json")

# Check if the file exists before attempting to read
if os.path.exists(input_file):
    # Read the results from the JSON file
    with open(input_file, 'r', encoding='utf-8') as json_file:
        ground_truth = json.load(json_file)  # Load the JSON data into a Python object

    # Now ground_truth contains the data from ground_truth_retrieval.json
    print(f"Data loaded from {input_file}")
    print(f"Number of entries: {len(ground_truth)}")
    # Optionally display the first few records
    print(ground_truth[:5])  # Display the first 5 entries
else:
    print(f"File {input_file} does not exist!")

Data loaded from /home/jovyan/reviews-assistant/data/ground_truth/ground_truth_retrieval.json
Number of entries: 105
[{'appid': '2322010', 'question': 'How well does God of War: Ragnarok perform on PC at launch?'}, {'appid': '2322010', 'question': 'What kind of PC specifications did the reviewer use while playing God of War: Ragnarok?'}, {'appid': '2322010', 'question': 'What resolution and settings were used to play God of War: Ragnarok according to the review?'}, {'appid': '2322010', 'question': 'Did the reviewer experience any performance issues such as stutter while playing God of War: Ragnarok?'}, {'appid': '2322010', 'question': "Is there any mention of the game's story or gameplay quality in the review?"}]


In [35]:
ground_truth[:1]

[{'appid': '2322010',
  'question': 'How well does God of War: Ragnarok perform on PC at launch?'}]

In [36]:
evaluate(ground_truth, question_hybrid)

  0%|          | 0/105 [00:00<?, ?it/s]

{'appid': '2322010', 'question': 'How well does God of War: Ragnarok perform on PC at launch?'}
{'appid': '2322010', 'review': "God of War Ragnarok could have been an unforgettable experience on PC, but the unnecessary and nonsensical PSN/Sony account requirements tarnish what should have been a flawless port, and breaks offline play on Steam Deck. Sony, if you're reading this, stop shackling amazing games with terrible DRM and account dependencies. Especially single player games! Let us enjoy the game we paid for without the extra hoops."}
True
{'appid': '2322010', 'question': 'What kind of PC specifications did the reviewer use while playing God of War: Ragnarok?'}


  5%|▍         | 5/105 [00:00<00:02, 42.96it/s]

{'appid': '2322010', 'review': "to God of War: Ragnarok Dev. Team,\n\nI’m writing to bring to your attention an issue regarding the current PC port of God of War: Ragnarok that affects users with 4GB VRAM graphics cards. When attempting to launch the game on such systems, an error message is displayed, indicating that there is not enough VRAM, and the game closes without allowing users to proceed. It is my humble request to kindly reconsider or remove this VRAM limitation. From personal experience and community feedback, I can confirm that the game runs well on certain integrated graphics processors (such as the Ryzen 3 3200G, Ryzen 5 3400G, and Ryzen 7 5700G), which are generally weaker than many 4GB VRAM GPUs available today. God of War: Ragnarok performs on the base PlayStation 4, which further suggests that the game should be capable of running on 4GB VRAM graphics cards, and the latest Steam Hardware Survey shows that one of the most widely used graphics cards among gamers is the 

 10%|▉         | 10/105 [00:00<00:02, 43.51it/s]

...

 97%|█████████▋| 102/105 [00:02<00:00, 42.91it/s]

{'appid': '1817070', 'review': "Spider-Man Remastered really takes an already amazing game and just makes it so much better. Visual improvements in this are stunning, and swinging through New York feels smoother than ever. Even the faster load times and updated character models give it a different feel, but it still retains all the fun of the original. It still retains a story that is just as gripping, and the combat is super fluid, satisfying. Honestly, a replay or jumping in for the first time, it is one of the best superhero games out there.\r\n\r\nJust incredible visual improvements, smoother mechanics, and faster load times take an already great game in Spider-Man Remastered and elevate the experience to a whole new level. The story remains compelling and emotionally engaging, while the combat and traversal mechanics continue to shine, offering a fluid and dynamic gameplay experience. However, the open world can feel a bit repetitive at times, and some stealth and puzzle sections 

100%|██████████| 105/105 [00:02<00:00, 44.74it/s]

{'appid': '794540', 'review': 'Giving it a thumbs up but with heavy reservations.\r\n\r\nNeo Cab has us take a role of Lina, a cab driver in a neo-futuristic city. She\'s just moved in to the city to live with her best friend from the past whom she hasn\'t seen in ages. However, on the first night, her friend goes missing and Lina takes it upon herself to find out what happened to her friend and who is responsible while at the same time trying to make a living by driving passengers.\r\n\r\nSo in a sense, Neo Cab is a hybrid of a visual novel story and management gameplay which involves serving customers, making enough money to get by, and making sure your star rating stays high. Normally, this kind of genre mixing can be risky, but what the game has is connected well enough, without either aspect getting in the way of the other. The story is giving us an end goal to strive towards, whereas the management aspect is giving us something to do and think about while we\'re progressing throu

{'hit_rate': 1.0, 'mrr': 1.0}

In [61]:
# {'appid': '1817070', 'question': "Is Marvel's Spider-Man Remastered a well-optimized game upon release?"}
# {'appid': '1817070', 'review': "This game is absolutely amazing, its extremely well optimized, great graphics, i dont get tired at looking the beautiful different suits, one of the best stories i've seen in videogames. If you like Spider-Man buy this game. If you liked Spider-Man just a little bit as a kid, like me, buy this game. Even if you don't care much about Spider-Man but like a well-written, solid game, buy this game."}
# True

In [62]:
query="Is Marvel's Spider-Man Remastered a well-optimized game upon release?"

In [65]:
appid="1817070"

In [68]:
v_q = model.encode(query)

In [71]:
results = indexer.query_reviews_hybrid(appid, query, v_q, vector_field="title_review_vector")

In [72]:
results

[{'appid': '1817070',
  'review': "Marvel's Spider-Man Remastered may not be as well-rounded an experience as Rocksteady's Arkham series. But when you're swinging through Time Square in glorious sunshine as New York's citizenry gazes at you from below, well, it's a superhero high that even the Caped Crusader would struggle to beat."},
 {'appid': '1817070',
  'review': "This game is absolutely amazing, its extremely well optimized, great graphics, i dont get tired at looking the beautiful different suits, one of the best stories i've seen in videogames. If you like Spider-Man buy this game. If you liked Spider-Man just a little bit as a kid, like me, buy this game. Even if you don't care much about Spider-Man but like a well-written, solid game, buy this game."}]

In [40]:
import json
import os

# Define the data directory path
data_dir = os.path.abspath('../reviews-assistant/data/ground_truth')

# Define the input file path
input_file = os.path.join(data_dir, "ground_truth_retrieval.json")

# Check if the file exists before attempting to read
if os.path.exists(input_file):
    # Read the results from the JSON file
    with open(input_file, 'r', encoding='utf-8') as json_file:
        ground_truth = json.load(json_file)  # Load the JSON data into a Python object

    # Now ground_truth contains the data from ground_truth_retrieval.json
    print(f"Data loaded from {input_file}")
    print(f"Number of entries: {len(ground_truth)}")
    # Optionally display the first few records
    print(ground_truth[:5])  # Display the first 5 entries
else:
    print(f"File {input_file} does not exist!")

Data loaded from /home/jovyan/reviews-assistant/data/ground_truth/ground_truth_retrieval.json
Number of entries: 105
[{'appid': '2322010', 'question': 'How well does God of War: Ragnarok perform on PC at launch?'}, {'appid': '2322010', 'question': 'What kind of PC specifications did the reviewer use while playing God of War: Ragnarok?'}, {'appid': '2322010', 'question': 'What resolution and settings were used to play God of War: Ragnarok according to the review?'}, {'appid': '2322010', 'question': 'Did the reviewer experience any performance issues such as stutter while playing God of War: Ragnarok?'}, {'appid': '2322010', 'question': "Is there any mention of the game's story or gameplay quality in the review?"}]


In [41]:
ground_truth[:2]

[{'appid': '2322010',
  'question': 'How well does God of War: Ragnarok perform on PC at launch?'},
 {'appid': '2322010',
  'question': 'What kind of PC specifications did the reviewer use while playing God of War: Ragnarok?'}]

In [42]:
from openai import OpenAI

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

In [43]:
def build_prompt(query, search_results):
    answer_llm = ""

    if 'entry_template' in globals():  # Check if entry_template exists in global scope
        for doc in search_results:
            answer_llm += entry_template.format(**doc) + "\n\n"
    else:
        for doc in search_results:
            # Fallback formatting if entry_template is missing
            answer_llm += str(doc) + "\n\n"

    prompt = prompt_template.format(question=query, answer_llm=answer_llm).strip()
    return prompt

In [44]:
def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [45]:
def rag(query, model='gpt-4o-mini'):
    # search_results = search(query)
    search_results = indexer.query_reviews_hybrid(appid, query, v_q, vector_field=vector_field)
    print(search_results)
    prompt = build_prompt(query, search_results)
    #print(prompt)
    answer = llm(prompt, model=model)
    # print(answer)
    return answer

In [46]:
prompt_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [47]:
evaluations = []

for record in tqdm(ground_truth):
    question = record['question']
    print(question)
    answer_llm = rag(question) 

    prompt = prompt_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt)
    evaluation = json.loads(evaluation)

    evaluations.append((record, answer_llm, evaluation))

  0%|          | 0/105 [00:00<?, ?it/s]

How well does God of War: Ragnarok perform on PC at launch?
[{'appid': '2322010', 'review': "to God of War: Ragnarok Dev. Team,\n\nI’m writing to bring to your attention an issue regarding the current PC port of God of War: Ragnarok that affects users with 4GB VRAM graphics cards. When attempting to launch the game on such systems, an error message is displayed, indicating that there is not enough VRAM, and the game closes without allowing users to proceed. It is my humble request to kindly reconsider or remove this VRAM limitation. From personal experience and community feedback, I can confirm that the game runs well on certain integrated graphics processors (such as the Ryzen 3 3200G, Ryzen 5 3400G, and Ryzen 7 5700G), which are generally weaker than many 4GB VRAM GPUs available today. God of War: Ragnarok performs on the base PlayStation 4, which further suggests that the game should be capable of running on 4GB VRAM graphics cards, and the latest Steam Hardware Survey shows that on

  1%|          | 1/105 [00:02<04:29,  2.59s/it]

What kind of PC specifications did the reviewer use while playing God of War: Ragnarok?
[{'appid': '2322010', 'review': "God of War Ragnarok could have been an unforgettable experience on PC, but the unnecessary and nonsensical PSN/Sony account requirements tarnish what should have been a flawless port, and breaks offline play on Steam Deck. Sony, if you're reading this, stop shackling amazing games with terrible DRM and account dependencies. Especially single player games! Let us enjoy the game we paid for without the extra hoops."}, {'appid': '2322010', 'review': "to God of War: Ragnarok Dev. Team,\n\nI’m writing to bring to your attention an issue regarding the current PC port of God of War: Ragnarok that affects users with 4GB VRAM graphics cards. When attempting to launch the game on such systems, an error message is displayed, indicating that there is not enough VRAM, and the game closes without allowing users to proceed. It is my humble request to kindly reconsider or remove thi

  2%|▏         | 2/105 [00:06<05:24,  3.16s/it]

...

 98%|█████████▊| 103/105 [04:18<00:04,  2.36s/it]

What themes are explored in Neo Cab through its narrative and character interactions?
[{'appid': '2322010', 'review': "[h1]A Great Sequel, Held Back by Woke[/h1]\n\n[h3]Introduction[/h3]\n\nI played God of War: Ragnarök earlier this year on my PS5, spending over 90 hours to get the platinum for both the base game and the Valhalla DLC. You can check my stats and trophies here from the comments. After all those hours of gameplay, I can confidently say that Ragnarök delivers a great experience, though it has some issues that hold it back from being a masterpiece like its predecessor.\n\n[h3]Not Just a DLC[/h3] \n \nOne of the biggest criticisms people have thrown at Ragnarök is that it feels like a glorified DLC of the 2018 game, but I strongly disagree with that take. Sure, it has a similar design and gameplay structure, but that doesn’t make it a mere DLC. There’s more than enough new content, including new realms, characters, and abilities, to call it a full-fledged sequel. The combat,

 99%|█████████▉| 104/105 [04:21<00:02,  2.29s/it]

Is there a built-in system for saving progress in Neo Cab, and how does it work?
[{'appid': '2322010', 'review': 'Level design is on top, multiple puzzles and great award system. Each location is filled with treasures and focal guides. They tell the story here.\n\nThe story telling itself is great. GoW 2018, compared to Ragnarök, felt more "hopeful" and positive in a way, whilst Ragnarök setting feels like desperation and end of times (as it should tho). Great work of Santa Monica Studio.\n\nSpoiler [spoiler] Not a fan playing as Atreus. He\'s got great moves and fun combat, but the Ironwood level was too long and boring in most parts. [/spoiler]\n\nAs for PSN account, I\'ve had one before, so it took me literally a few seconds to log in. Not sure how the process goes for those who do not have any accounts, but the fact that it is required in order to play a singleplayer game sucks.\n\nGreat game tho.'}, {'appid': '2322010', 'review': 'PSN account is mandatory with no option to skip. R

100%|██████████| 105/105 [04:23<00:00,  2.51s/it]


In [49]:
df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])

df_eval['appid'] = df_eval.record.apply(lambda d: d['appid'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['explanation'])

del df_eval['record']
del df_eval['evaluation']

In [50]:
df_eval.relevance.value_counts()

relevance
NON_RELEVANT       72
PARTLY_RELEVANT    23
RELEVANT           10
Name: count, dtype: int64

In [51]:
df_eval.relevance.value_counts(normalize=True)

relevance
NON_RELEVANT       0.685714
PARTLY_RELEVANT    0.219048
RELEVANT           0.095238
Name: proportion, dtype: float64

In [52]:
df_eval[df_eval["relevance"]=="RELEVANT"]

,answer,appid,question,relevance,explanation
4,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",2322010,Is there any mention of the game's story or ga...,RELEVANT,The generated answer clearly indicates that bo...
18,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",582160,What were the standout features mentioned in t...,RELEVANT,The generated answer explicitly addresses the ...
20,"{\n ""relevance"": ""PARTLY_RELEVANT"",\n ""expla...",447040,Does Watch_Dogs 2 have any major bugs or perfo...,RELEVANT,The generated answer directly addresses the qu...
33,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",315210,"Was the game released in a polished state, or ...",RELEVANT,The generated answer clearly addresses the que...
46,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",1680880,How did the reviewer feel about the game's per...,RELEVANT,The generated answer effectively summarizes th...
57,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",1832040,Does the game experience any performance issue...,RELEVANT,The generated answer directly addresses the qu...
61,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",2443720,What is the game's performance on high-end har...,RELEVANT,The generated answer accurately addresses the ...
74,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",1086940,What are the reviews saying about the game's p...,RELEVANT,The generated answer directly addresses the qu...
84,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",2698940,Is The Crew Motorfest worth buying at full pri...,RELEVANT,The generated answer addresses the question di...
93,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",1496790,"What are the reported performance issues, spec...",RELEVANT,The generated answer directly addresses the qu...


In [53]:
df_eval[df_eval["relevance"]=="NON_RELEVANT"]

,answer,appid,question,relevance,explanation
1,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2322010,What kind of PC specifications did the reviewe...,NON_RELEVANT,The answer does not address the specific quest...
2,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2322010,What resolution and settings were used to play...,NON_RELEVANT,The generated answer does not provide any info...
3,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2322010,Did the reviewer experience any performance is...,NON_RELEVANT,The generated answer does not address the spec...
5,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2369390,"Is the storytelling in Far Cry 6 engaging, or ...",NON_RELEVANT,The generated answer does not address the ques...
7,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2369390,How does the story in Far Cry 6 compare to nar...,NON_RELEVANT,The generated answer does not provide any info...
...,...,...,...,...,...
100,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",794540,Is Neo Cab a fully optimized game that functio...,NON_RELEVANT,The generated answer completely deviates from ...
101,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",794540,Does Neo Cab deliver on the promises made by i...,NON_RELEVANT,The generated answer fails to address the ques...
102,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",794540,How does the dialogue in Neo Cab contribute to...,NON_RELEVANT,The generated answer fails to discuss the dial...
103,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",794540,What themes are explored in Neo Cab through it...,NON_RELEVANT,The generated answer completely fails to addre...


In [54]:
df_eval[df_eval["appid"]=="2322010"]

,answer,appid,question,relevance,explanation
0,"{\n ""relevance"": ""PARTLY_RELEVANT"",\n ""expla...",2322010,How well does God of War: Ragnarok perform on ...,PARTLY_RELEVANT,The generated answer acknowledges some aspects...
1,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2322010,What kind of PC specifications did the reviewe...,NON_RELEVANT,The answer does not address the specific quest...
2,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2322010,What resolution and settings were used to play...,NON_RELEVANT,The generated answer does not provide any info...
3,"{\n ""relevance"": ""NON_RELEVANT"",\n ""explanat...",2322010,Did the reviewer experience any performance is...,NON_RELEVANT,The generated answer does not address the spec...
4,"{\n ""relevance"": ""RELEVANT"",\n ""explanation""...",2322010,Is there any mention of the game's story or ga...,RELEVANT,The generated answer clearly indicates that bo...


In [55]:
import json
import os
import pandas as pd

# Sample DataFrame for demonstration
# df_eval = pd.DataFrame({'recommendationid': [1, 2], 'question': ['What is the best game?', 'What is the worst game?']})

data_dir = os.path.abspath('../reviews-assistant/data/ground_truth')

# Define the output file path
output_file = os.path.join(data_dir, "ground_truth_evaluation.json")
6
# Convert DataFrame to a list of dictionaries including index
df_eval_list = df_eval.reset_index().to_dict(orient='records')

# Save the results to a JSON file
with open(output_file, 'w', encoding='utf-8') as json_file:
    json.dump(df_eval_list, json_file, ensure_ascii=False, indent=4)

print(f"Data saved to {output_file}")

Data saved to /home/jovyan/reviews-assistant/data/ground_truth/ground_truth_evaluation.json


In [56]:
reviews[0]

{'appid': '812140',
 'timestamp_query': 1727643597,
 'title': "Assassin's Creed Odyssey",
 'recommendationid': '174160338',
 'author.steamid': '76561198420943538',
 'author.playtimeforever': 0,
 'author.playtime_last_two_weeks': 0,
 'author.playtime_at_review': 5599,
 'author.last_played': 1696845057,
 'language': 'english',
 'review': "---{ Graphics }---\n✅ You forget what reality is\n☐ Beautiful\n☐ Good\n☐ Decent\n☐ Bad\n☐ You will get eye cancer\n☐ Get a pepper spray for your eye instead\n\n---{ Gameplay }---\n☐ Won’t ever touch any other game anymore\n✅ Very good\n☐ Good\n☐ It's just gameplay\n☐ Mehh\n☐ Watch paint dry instead\n☐ Tic Tac toe is better\n\n---{ Audio }---\n☐ Eargasm\n✅ Very good\n☐ Good\n☐ Not too bad\n☐ Bad\n☐ I'm now deaf\n\n---{ Audience }---\n☐ Kids\n✅Teens\n✅ Adults\n☐ Grandma\n\n---{ PC Requirements }---\n☐ Check if you can run paint\n☐ Potato\n☐ Decent\n✅ Fast\n☐ Rich boi\n☐ Ask NASA if they have a spare computer\n☐ Search the galaxy for dark matter fuel to ru

In [57]:
reviews[-1]

{'appid': '2239550',
 'timestamp_query': 1727643597,
 'title': 'Watch Dogs: Legion',
 'recommendationid': '174862159',
 'author.steamid': '76561199371555316',
 'author.playtimeforever': 0,
 'author.playtime_last_two_weeks': 1,
 'author.playtime_at_review': 109,
 'author.last_played': 1726993288,
 'language': 'english',
 'review': 'meh',
 'voted_up': True,
 'votes_up': 0,
 'timestamp_created': 1726233314,
 'timestamp_updated': 1726233314,
 'title_vector': [-0.031208040192723274,
  -0.04140456020832062,
  0.007626562379300594,
  -0.05640276148915291,
  0.017036626115441322,
  0.07028614729642868,
  -0.02655501663684845,
  -0.03512350469827652,
  0.05046771839261055,
  -0.004683449864387512,
  0.0064065903425216675,
  -0.034316763281822205,
  0.06852371990680695,
  -0.0006094787968322635,
  0.007349507883191109,
  -0.016963135451078415,
  0.0868942141532898,
  0.009798023849725723,
  0.03648601472377777,
  -0.01720307394862175,
  -0.06752648204565048,
  -0.03290846198797226,
  0.010983460